# MSTR - Robotics: Schema Monitor

### Start Notebook

In [42]:
from mstr_robotics.mstr_classes import  get_conn ,mstr_global
import pandas as pd
import json
from mstr_robotics._helper import msic,str_func 
#from mstrio.api import objects as api_obj
from mstr_robotics.mstr_pandas import df_helper
from mstr_robotics.report import cube,rep
from mstr_robotics.read_out_prj_obj import read_schema,chk_usage,io_attributes


i_msic=msic()
i_rep=rep()
i_str_func=str_func()
i_cube=cube()
i_chk_usage=chk_usage()
i_df_helper=df_helper()
i_read_schema=read_schema()
run_prop_d={}


#in my environment the PA project is on the same maschine

username = "Administrator"
password =""
server="your_ip"
port="your_port"
#project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
project_id="49EA696942D930ACE898D0BD05A8AF4E"

base_url= "http://" + server + ":" + port + "/MicroStrategyLibrary/api"
pa_base_url= "http://" + server + ":" + port + "/MicroStrategyLibrary/api"

conn=get_conn(base_url=base_url,username=username,password=password)
conn.select_project(project_id)
           
pa_conn=get_conn(base_url=pa_base_url,username=username,password=password)

Connection to MicroStrategy Intelligence Server has been established.
No project selected.
Connection to MicroStrategy Intelligence Server has been established.
No project selected.


### Set id's for underlying cubes

In [36]:
#Schema information
cbe_tbl_att_fct_id = "226F8DFD4CBF59123D91C5A28E5CB27F"
cbe_att_form_exp_id = "0C87612143983878A20FB4868B39666D"
cbe_fact_exp_id = "8B40E68E45EC34D16232CEAE7A22789E"
cbe_table_id = "DAB859BE48E2CB2A7D0D84AA796C1DBC"

# blended object and PA/EM information
cbe_obj_depn_rep_col_usage_id = "E105F5BE4FA52CAEE264648069D978B5"

### read out schema table mappings

In [37]:
#read out the schema & builds a cube upload file for attributes, facts & table definitions

schema_mappings_d=i_read_schema.table_mappings(conn=conn,run_prop_d=run_prop_d)
att_form_exp_df=schema_mappings_d["att_form_exp_df"]
fact_exp_df=schema_mappings_d["fact_exp_df"]
table_df=schema_mappings_d["table_df"]
att_form_exp_d_l=[{"df":schema_mappings_d["att_form_exp_df"],"tbl_name":"att_form_exp_df", "update_policy":"Replace"}]
fact_exp_d_l=[{"df":schema_mappings_d["fact_exp_df"],"tbl_name":"fact_exp_df", "update_policy":"Replace"}]
table_d_l=[{"df":schema_mappings_d["table_df"],"tbl_name":"table_df", "update_policy":"Replace"}]

'id'


In [ ]:
#some times it's easier to join the datasets outside of a dossier
#here we build a primary key data set by joinning dataframes
tbl_att_fct_df = pd.merge(table_df[["project_id","table_id", "column_id","physicalTable_id","physicalTable_name"]],
                          att_form_exp_df[["project_id", 'table_id', "column_id", "attribute_id", "form_expressionId"]],
                          on=['project_id', 'table_id', 'column_id'], how='left')
i_df_helper.clean_double_col(df=tbl_att_fct_df)
tbl_att_fct_df = pd.merge(tbl_att_fct_df, 
                          fact_exp_df[["project_id", "column_id","fact_id","fact_expressionId"]],
                          on=['project_id', 'column_id'], how='left')
i_df_helper.clean_double_col(df=tbl_att_fct_df)

tbl_att_fct_d_l=[{"df":schema_mappings_d["tbl_att_fct_df"],"tbl_name":"tbl_att_fct_df", "update_policy":"Replace"}]

## report usage

### Platform Analytics

In [ ]:
pa_project_id="01770E1B45A0B84E88E5748B465719AD"
pa_conn.select_project(pa_project_id)

In [ ]:
#report stats
report_usage="BF03DD1443947C6192D0229B4A3D7EF8"

instance_id=i_rep.open_Instance(conn=pa_conn,report_id=report_usage)
report_usage_df=i_rep.rep_to_dataframe(conn=pa_conn,report_id=report_usage,instance_id=instance_id)
adjust_col_names_to_em = {"project_id@ID":"project_id", "report_id@ID":"report_id", "year_id@ID":"year_id"
                          , "month_id@ID":"month_id"}
report_usage_df.rename(columns=adjust_col_names_to_em, inplace=True)
rep_usage_d_l=[{"df":rep_usage_df,"tbl_name":"rep_usage_df", "update_policy":"Replace"}]

In [ ]:
# db views & column stats
views_cols_usage="9A96A6474DB13EA45D4B96B4485463C8"
instance_id=i_rep.open_Instance(conn=pa_conn,report_id=views_cols_usage)
view_col_usage_df=i_rep.rep_to_dataframe(conn=pa_conn,report_id=views_cols_usage,instance_id=instance_id)
adjust_col_names_to_em = {"project_id@ID":"project_id", "report_id@ID":"report_id", "year_id@ID":"year_id"
                          , "month_id@ID":"month_id", "db_table@GUID":"physicalTable_id", 
                          "column@Name":"column_name","column@GUID":"column_id"}
view_col_usage_df.rename(columns=adjust_col_names_to_em, inplace=True)
view_col_usage_d_l=[{"df":view_col_usage_df,"tbl_name":"view_col_usage_df", "update_policy":"Replace"}]

### Enterprise Manager Statistic tables

In [39]:
# report stats AND db views & column stats

project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn.select_project(project_id)
views_cols_usage="9F5167F6406AA0E0BC72D7A8141CED4C"
report_usage="C1F2185F4610B81774334788348A8F31"

instance_id=i_rep.open_Instance(conn=conn,report_id=views_cols_usage)
view_col_usage_df=i_rep.rep_to_dataframe(conn=conn,report_id=views_cols_usage,instance_id=instance_id)
view_col_usage_df=i_df_helper.rem_att_id_forms(view_col_usage_df)

instance_id=i_rep.open_Instance(conn=conn,report_id=report_usage)
rep_usage_df=i_rep.rep_to_dataframe(conn=conn,report_id=report_usage,instance_id=instance_id)
rep_usage_df=i_df_helper.rem_att_id_forms(rep_usage_df)

rep_usage_d_l=[{"df":rep_usage_df,"tbl_name":"rep_usage_df", "update_policy":"Replace"}]
view_col_usage_d_l=[{"df":view_col_usage_df,"tbl_name":"view_col_usage_df", "update_policy":"Replace"}]

## Object Information 

### REST API search

In [40]:

#advantage is very flexible & includes security
#disadvantage: takes ages!!!!!

obj_type_l=["3"] #only check reports
obj_depn_df =i_chk_usage.chk_by_obj_type(conn=conn #,project_id=project_id
                          ,obj_type_l=obj_type_l, info_level = "base_path",mtdi_id=None
                          ,count_only_fg=True,run_prop_d=run_prop_d)

obj_depn_df.rename(columns={'id': 'report_id'}, inplace=True)
obj_depn_d_l=[{"df":obj_depn_df,"tbl_name":"obj_depn_df", "update_policy":"Replace"}]

#i_cube.upload_data(conn=conn,tbl_upd_dict=obj_depn_d_l, mtdi_id=cbe_obj_depn_id,cube_name="obj_depn",folder_id=folder_id,force=True)

### from sql server MD

In [ ]:
#for the sample dossier pls. do not rund this
#sql_server_md_free_form_rep="AB4F44E441A74CFD1FFFE2A348769E56"
#instance_id=i_rep.open_Instance(conn=conn,report_id=sql_server_md_free_form_rep)
#sql_server_md_free_form_rep_df=i_rep.rep_to_dataframe(conn=conn,report_id=sql_server_md_free_form_rep,instance_id=instance_id)
#obj_depn_d_l=[{"df":sql_server_md_free_form_rep_df,"tbl_name":"obj_depn_df", "update_policy":"Replace"}]

### Flag objects

In [41]:
#in this case on folder
#this feature is based on the column path_ids with is a simple string concatinating the
# all parent folder_guids in the followinf style


flag_path_d_l=[{"in_duty":"Daniel",
                "flag_folder_id":"032A5E114A59D28267BDD8B6D9E58B22"},
              {"in_duty":"Verena",
                "flag_folder_id": "F025A94B4C03B6DCEE0F5D9DA825DA67"},
              {"in_duty":"Dolores",
                "flag_folder_id":"92ADD0F84D07AC532AD03BA0F92A836B"},
              {"in_duty":"Ed",
                "flag_folder_id": "7F16A4B811E58ED317D50080EFF554EA"},
                {"in_duty":"Mike",
                "flag_folder_id": "D64C532E4E7FBA74D29A7CA3576F39CF"}]

depn_obj_df_flag=i_df_helper.flag_folder_df(flag_path_d_l=flag_path_d_l,obj_df=obj_depn_df,path="path_ids"
                                            ,flag_folder_id="flag_folder_id", merge_key_l=['project_id', 'report_id'])
obj_depn_d_l=[{"df":depn_obj_df_flag,"tbl_name":"obj_depn_df", "update_policy":"Replace"}]

## Cubing

### load cubes

In [43]:
# if the provided mtdi_id = None a new cube is crated
# if a valid mtdi_id is provided an existing cube is updated
# if the mtdi_id does not exists 
# or the columns of the dataframe do not fit to the cube
# you get an error msg
cube_folder_id="80CF2C7648B4E9A93C82BD95892E0060" #Schema Monitor

#primery key cube schema usage
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=tbl_att_fct_d_l,mtdi_id=cbe_tbl_att_fct_id , cube_name="tbl_att_fct",folder_id=cube_folder_id,force=True)
#attribute form expressions 
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=att_form_exp_d_l,mtdi_id=cbe_att_form_exp_id , cube_name="att_form_exp",folder_id=cube_folder_id,force=True)
#fact expressions
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=fact_exp_d_l,mtdi_id=cbe_fact_exp_id , cube_name="fact_exp",folder_id=cube_folder_id,force=True)
#table definitions
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=table_d_l,mtdi_id=cbe_table_id , cube_name="table",folder_id=cube_folder_id,force=True)

#define multi table cube
#ensure that derived attributes are mapped 
#over common column names
obj_depn_rep_col_usage_l=[]
obj_depn_rep_col_usage_l.extend(obj_depn_d_l)
obj_depn_rep_col_usage_l.extend(rep_usage_d_l)
obj_depn_rep_col_usage_l.extend(view_col_usage_d_l)
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_depn_rep_col_usage_l, mtdi_id=cbe_obj_depn_rep_col_usage_id,cube_name="obj_depn_rep_col_usage",folder_id=cube_folder_id,force=True)
conn.close()
#pa_conn.close()

SuperCube object named: 'tbl_att_fct' with ID: '226F8DFD4CBF59123D91C5A28E5CB27F'


  0%|          | 0/1 [00:00<?, ?it/s]

Super cube 'tbl_att_fct' published successfully.
SuperCube object named: 'att_form_exp' with ID: '0C87612143983878A20FB4868B39666D'


  0%|          | 0/1 [00:00<?, ?it/s]

Super cube 'att_form_exp' published successfully.
SuperCube object named: 'fact_exp' with ID: '8B40E68E45EC34D16232CEAE7A22789E'


...

Super cube 'table' published successfully.
SuperCube object named: 'obj_depn_rep_col_usage' with ID: 'E105F5BE4FA52CAEE264648069D978B5'


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Super cube 'obj_depn_rep_col_usage' published successfully.
Connection to MicroStrategy Intelligence Server has been closed.
